In [256]:
import pandas as pd
from openpyxl.workbook import Workbook
#pd.options.display.max_rows=12

In [257]:
#load excel file into pandas dataframe
df = pd.read_excel('AlzheimerData.xlsx', skiprows=1)
#original dataframe
df

,Y,X1,X2,X3,X4,X5,X6,X7,X8,X9,X10,X11,X12,X13,X14,X15
0,517.06,9.233,1.130,0,7,16,4,0.067,4,0,12,2,!!!,39.3,3.6,7.1
1,504.06,9.001,1.129,0,8,16,4,0.068,4,0,12,1,0.063,39.3,1.8,7.1
2,490.03,9.246,1.128,0,8,16,3,0.054,4,0,12,1,0.074,39.6,1.9,7.5
3,1443.08,6.414,1.146,0,!!!,20,42,0.187,sds,0,6,qwe,0.069,wqew,^^^^,7.6
4,517.06,9.233,1.130,0,7,16,4,0.067,4,0,12,2,$$$,39.3,3.6,7.1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20,533.11,8.74,1.133,0,10,16,4,0.062,4,0,NaN,2,4,37.7,3.3,6.6
21,477.50,8.527,1.128,0,5,21,5,0.083,3,0,18,3,4,46.4,5.4,7.1
22,548.59,8.312,1.132,0,7,22,7,QQQQ,4,0,18,4,5,43.9,6.1,7.6
23,511.72,7.207,1.145,0,6,9,5,0.067,3,0,6,3,2,36.6,4.2,2.8


In [259]:
#Remove all columns with zero value
df = df.loc[:, (df != 0).any(axis=0)]
df

,Y,X1,X2,X4,X5,X6,X7,X8,X10,X11,X12,X13,X14,X15
0,517.06,9.233,1.130,7,16,4,0.067,4,12,2,!!!,39.3,3.6,7.1
1,504.06,9.001,1.129,8,16,4,0.068,4,12,1,0.063,39.3,1.8,7.1
2,490.03,9.246,1.128,8,16,3,0.054,4,12,1,0.074,39.6,1.9,7.5
3,1443.08,6.414,1.146,!!!,20,42,0.187,sds,6,qwe,0.069,wqew,^^^^,7.6
4,517.06,9.233,1.130,7,16,4,0.067,4,12,2,$$$,39.3,3.6,7.1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20,533.11,8.74,1.133,10,16,4,0.062,4,NaN,2,4,37.7,3.3,6.6
21,477.50,8.527,1.128,5,21,5,0.083,3,18,3,4,46.4,5.4,7.1
22,548.59,8.312,1.132,7,22,7,QQQQ,4,18,4,5,43.9,6.1,7.6
23,511.72,7.207,1.145,6,9,5,0.067,3,6,3,2,36.6,4.2,2.8


In [260]:
#check type of each coulum and then count of coulmn for each type 
df.dtypes.value_counts()

object     12
float64     2
dtype: int64

In [261]:
#dfo is the part of dataframe that contain garbage characters and should be clean
dfo=df.select_dtypes('object')

dfo

,X1,X4,X5,X6,X7,X8,X10,X11,X12,X13,X14,X15
0,9.233,7,16,4,0.067,4,12,2,!!!,39.3,3.6,7.1
1,9.001,8,16,4,0.068,4,12,1,0.063,39.3,1.8,7.1
2,9.246,8,16,3,0.054,4,12,1,0.074,39.6,1.9,7.5
3,6.414,!!!,20,42,0.187,sds,6,qwe,0.069,wqew,^^^^,7.6
4,9.233,7,16,4,0.067,4,12,2,$$$,39.3,3.6,7.1
...,...,...,...,...,...,...,...,...,...,...,...,...
20,8.74,10,16,4,0.062,4,NaN,2,4,37.7,3.3,6.6
21,8.527,5,21,5,0.083,3,18,3,4,46.4,5.4,7.1
22,8.312,7,22,7,QQQQ,4,18,4,5,43.9,6.1,7.6
23,7.207,6,9,5,0.067,3,6,3,2,36.6,4.2,2.8


In [262]:
#show total number of each type in the part of dataframe containig garbage(dfo :the part that we want to clean). 
dfo.applymap(type).stack().value_counts()

<class 'int'>      159
<class 'float'>    121
<class 'str'>       20
dtype: int64

In [264]:
#Parse function: input x(cell of the daraframe), return True if length of x equal and greater than 5 and False if less than 5
# this function is used to check whether we should drop a row or coulmn(if there is a string more than 5, they should be dropped)
def parse(x):
    if type(x)== str:
        import re
        x_str=re.sub(x,'[0-9]','')
        if len(x_str)>=5:
            return  True
    return False

#Remove all rows and columns if they have string more than 5 by the use of pars function to check it
#dfo2 keep rows and columns of dfo that all cells of that row or column contain either int or float or string less than 5
#So when we apply dfo.applymap(parse) all cells get false or 0,(for rows sum(1)==0) and for coulumn sum(0)==0)
dfo2= dfo.loc[dfo.applymap(parse).sum(1)==0,dfo.applymap(parse).sum(0)==0]

dfo2

,X1,X4,X5,X6,X8,X10,X11,X13,X15
0,9.233,7,16,4,4,12,2,39.3,7.1
1,9.001,8,16,4,4,12,1,39.3,7.1
2,9.246,8,16,3,4,12,1,39.6,7.5
5,8.3,7,23,5,5,18,4,46.8,4.8
6,8.781,8,16,5,4,12,1,39,6.8
...,...,...,...,...,...,...,...,...,...
20,8.74,10,16,4,4,NaN,2,37.7,6.6
21,8.527,5,21,5,3,18,3,46.4,7.1
22,8.312,7,22,7,4,18,4,43.9,7.6
23,7.207,6,9,5,3,6,3,36.6,2.8


In [265]:
#Parse_replace: replce each cell of dataframe by float number except for  error
# for string except raise and replace string with 0
#these strings should be less than 5 because all string equal and greater than 5 have alrealy been deleted in previous part
def parse_replace(x):
    try:
        return float(x)
    except:
        return 0

df_clean = dfo2.applymap(parse_replace)   
df_clean

,X1,X4,X5,X6,X8,X10,X11,X13,X15
0,9.233,7.0,16.0,4.0,4.0,12.0,2.0,39.3,7.1
1,9.001,8.0,16.0,4.0,4.0,12.0,1.0,39.3,7.1
2,9.246,8.0,16.0,3.0,4.0,12.0,1.0,39.6,7.5
5,8.300,7.0,23.0,5.0,5.0,18.0,4.0,46.8,4.8
6,8.781,8.0,16.0,5.0,4.0,12.0,1.0,39.0,6.8
...,...,...,...,...,...,...,...,...,...
20,8.740,10.0,16.0,4.0,4.0,NaN,2.0,37.7,6.6
21,8.527,5.0,21.0,5.0,3.0,18.0,3.0,46.4,7.1
22,8.312,7.0,22.0,7.0,4.0,18.0,4.0,43.9,7.6
23,7.207,6.0,9.0,5.0,3.0,6.0,3.0,36.6,2.8


In [266]:
#Clean part of dataframe that exclude from dirty part of dataframe in the the begining
df_exclude = df.select_dtypes(exclude='object')
df_exclude

,Y,X2
0,517.06,1.130
1,504.06,1.129
2,490.03,1.128
3,1443.08,1.146
4,517.06,1.130
...,...,...
20,533.11,1.133
21,477.50,1.128
22,548.59,1.132
23,511.72,1.145


In [267]:
#Concat dfo2(the part of dataframe that got cleaned) with the rest clean part of data frame and fill nan values with zero
df2=pd.concat([df_exclude,df_clean],1).fillna(0)
#reorder column order based on original column order of dataframe
cols = list(df2.columns)
cols
a, b = cols.index('X1'), cols.index('X2')
cols[b], cols[a] = cols[a], cols[b]
df2 = df2[cols]

#final clean dataframe
df2    

,Y,X1,X2,X4,X5,X6,X8,X10,X11,X13,X15
0,517.06,9.233,1.130,7.0,16.0,4.0,4.0,12.0,2.0,39.3,7.1
1,504.06,9.001,1.129,8.0,16.0,4.0,4.0,12.0,1.0,39.3,7.1
2,490.03,9.246,1.128,8.0,16.0,3.0,4.0,12.0,1.0,39.6,7.5
3,1443.08,0.000,1.146,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,517.06,0.000,1.130,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...
20,533.11,8.740,1.133,10.0,16.0,4.0,4.0,0.0,2.0,37.7,6.6
21,477.50,8.527,1.128,5.0,21.0,5.0,3.0,18.0,3.0,46.4,7.1
22,548.59,8.312,1.132,7.0,22.0,7.0,4.0,18.0,4.0,43.9,7.6
23,511.72,7.207,1.145,6.0,9.0,5.0,3.0,6.0,3.0,36.6,2.8


In [268]:
#Calulate standardization( transforms data to have a mean of zero and a standard deviation of 1) 
#Calculate normalization(scale a variable to have a values between 0 and 1)
x_standardization=lambda x: (x-x.mean())/x.std()
x_normalization=lambda x: (x-x.mean())/(x.max()-x.min())
#df2.apply(x_standardization).T.dropna(how='all').T

In [275]:
#apply x_standardization to clean dataframe (df2)
standardization = df2.loc[:,df2.std()>0].apply(x_standardization)
standardization



,Y,X1,X2,X4,X5,X6,X8,X10,X11,X13,X15
0,-0.174260,0.626318,-0.132721,0.302257,0.215581,-0.019312,0.463492,0.156863,0.000000,0.216137,0.703945
1,-0.243789,0.554829,-0.247136,0.662088,0.215581,-0.019312,0.463492,0.156863,-0.738549,0.216137,0.703945
2,-0.318826,0.630324,-0.361550,0.662088,0.215581,-0.502117,0.463492,0.156863,-0.738549,0.237438,0.852144
3,4.778422,-2.218763,1.697914,-2.216554,-1.940226,-1.950530,-2.433335,-1.653096,-1.477098,-2.574332,-1.926587
4,-0.174260,-2.218763,-0.132721,-2.216554,-1.940226,-1.950530,-2.433335,-1.653096,-1.477098,-2.574332,-1.926587
...,...,...,...,...,...,...,...,...,...,...,...
20,-0.088419,0.474404,0.210523,1.381748,0.215581,-0.019312,0.463492,-1.653096,0.000000,0.102530,0.518696
21,-0.385841,0.408770,-0.361550,-0.417403,0.889270,0.463492,-0.260714,1.061843,0.738549,0.720268,0.703945
22,-0.005626,0.342519,0.096108,0.302257,1.024008,1.429101,0.463492,1.061843,1.477098,0.542757,0.889194
23,-0.202820,0.002021,1.583499,-0.057573,-0.727585,0.463492,-0.260714,-0.748117,0.738549,0.024425,-0.889194


In [276]:
#apply x_normalization to the clean dataframe(df2)
normalization = df2.loc[:,df2.std()>0].apply(x_normalization)
normalization

,Y,X1,X2,X4,X5,X6,X8,X10,X11,X13,X15
0,-0.033743,0.215245,-0.026977,0.084,0.069565,-0.005714,0.128,0.057778,0.00,0.064084,0.240506
1,-0.047207,0.190677,-0.050233,0.184,0.069565,-0.005714,0.128,0.057778,-0.25,0.064084,0.240506
2,-0.061737,0.216622,-0.073488,0.184,0.069565,-0.148571,0.128,0.057778,-0.25,0.070400,0.291139
3,0.925286,-0.762516,0.345116,-0.616,-0.626087,-0.577143,-0.672,-0.608889,-0.50,-0.763284,-0.658228
4,-0.033743,-0.762516,-0.026977,-0.616,-0.626087,-0.577143,-0.672,-0.608889,-0.50,-0.763284,-0.658228
...,...,...,...,...,...,...,...,...,...,...,...
20,-0.017121,0.163037,0.042791,0.384,0.069565,-0.005714,0.128,-0.608889,0.00,0.030400,0.177215
21,-0.074714,0.140481,-0.073488,-0.116,0.286957,0.137143,-0.072,0.391111,0.25,0.213558,0.240506
22,-0.001090,0.117713,0.019535,0.084,0.330435,0.422857,0.128,0.391111,0.50,0.160926,0.303797
23,-0.039274,0.000695,0.321860,-0.016,-0.234783,0.137143,-0.072,-0.275556,0.25,0.007242,-0.303797


In [277]:
standardization.to_csv("Lab2-Q2-Rescaled.txt", header=True, index=False, sep='\t', mode='a')
normalization.to_csv("Lab2-Q2-Normalized.txt",  header=True, index=False, sep='\t', mode='a')
df2.to_csv("Final-Clean-Data.txt",  header=True, index=False, sep='\t', mode='a')
